# Predicting March Madness

The goal of this project is to use multiple different machine learning methods on similar data to predict the results of the NCAA tournament.

# Imports

The dataset I'll be using comes from Kaggle's March Madness Mania 2023 challenge: https://www.kaggle.com/competitions/mens-march-mania-2023/data. The specific tables I'm using look at teams' rankings in different systems, each team's conference, their seed in the tournament, and each regular season result. I am going to limit my dataset to 2010's season and onward as 3-point shooting had not developed to its modern form until recently.

In [71]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [63]:
ordinals = pd.read_csv("MMasseyOrdinals.csv")
ordinals = ordinals[(ordinals["RankingDayNum"] == 133) & (ordinals["Season"]>2009)]
ordinals = ordinals.drop("RankingDayNum", axis=1)
ordinals.head(1)

,Season,SystemName,TeamID,OrdinalRank
1239532,2010,AP,1124,19


In [49]:
tourney = pd.read_csv("MNCAATourneyCompactResults.csv")
tourney = tourney.drop(["WLoc", "NumOT", "DayNum"], axis=1)
tourney = tourney[tourney["Season"]>2009]
tourney.head(1)

,Season,WTeamID,WScore,LTeamID,LScore
1584,2010,1115,61,1457,44


# Investigating Ordinal Ranks

I would like to simplify my model by either selecting only a single ordinal ranking system or by creating my own aggregate ranking based on multiple ranking systems.

In [59]:
def simple_df_transform(df):
    """Input a dataframe with columns 'Season', 'WTeamID', 'WScore', 'LTeamID', and 'LScore'.
    Return a new dataframe with 'Season', 'Team1', 'Team2', and 'Result' (0 if Team1 lost, 1 if Team1 won).
    The resulting dataframe will evenly and randomly split the data so half are losses and half are wins."""
    #Create the structure of the new dataframe (all wins for now)
    new_df = pd.DataFrame()
    new_df["Season"] = df["Season"]
    new_df["Team1"] = df["WTeamID"]
    new_df["Team2"] = df["LTeamID"]
    new_df["Result"] =  1
    
    #sample half of games to become losses
    losses = new_df.sample(frac=0.5)
    drop_losses = losses.index
    new_df = new_df.drop(drop_losses)
    
    #create a dataframe of losses with same structure as wins
    losses_df = pd.DataFrame()
    losses_df["Season"] = losses["Season"]
    losses_df["Team1"] = losses["Team2"]
    losses_df["Team2"] = losses["Team1"]
    losses_df["Result"] = 0
    
    #return concatenated loss and win dataframes
    return pd.concat([new_df, losses_df]).sort_index()

In [60]:
tourney_df = simple_df_transform(tourney)
tourney_df.head(1)

,Season,Team1,Team2,Result
1584,2010,1115,1457,1


In [76]:
#merge Team1 IDs
print(len(tourney_df))
ordinals1 = ordinals.rename(columns = {"TeamID": "Team1"})
team1_merged = tourney_df.merge(ordinals1, how="outer", on=["Season", "Team1"])
team1_merged.rename(columns = {"SystemName": "SystemName1", "OrdinalRank": "OrdinalRank1"}, inplace=True)
print(len(team1_merged))

#merge Team2 IDs
ordinals2 = ordinals.rename(columns = {"TeamID": "Team2"})
games_and_ranks = team1_merged.merge(ordinals2, how="outer", on=["Season", "Team2"])
games_and_ranks.rename(columns = {"SystemName": "SystemName2", "OrdinalRank": "OrdinalRank2"}, inplace=True)
print(len(games_and_ranks))
games_and_ranks.head(2)

800
261860
3387178


,Season,Team1,Team2,Result,SystemName1,OrdinalRank1,SystemName2,OrdinalRank2
0,2010,1115.0,1457.0,1.0,BIH,224.0,BIH,203.0
1,2010,1115.0,1457.0,1.0,BIH,224.0,BOB,198.0


In [82]:
#calculate difference in rank for each game
ordinal_tourney = games_and_ranks[games_and_ranks["SystemName1"] == games_and_ranks["SystemName2"]]
ordinal_tourney["RankDiff"] = ordinal_tourney["OrdinalRank1"] - ordinal_tourney["OrdinalRank2"]
ordinal_tourney.head(2)

/var/folders/dy/02vq48tn28l7g1y86391j3300000gn/T/ipykernel_13019/62555471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordinal_tourney["RankDiff"] = ordinal_tourney["OrdinalRank1"] - ordinal_tourney["OrdinalRank2"]


,Season,Team1,Team2,Result,SystemName1,OrdinalRank1,SystemName2,OrdinalRank2,RankDiff
0,2010,1115.0,1457.0,1.0,BIH,224.0,BIH,203.0,21.0
49,2010,1115.0,1457.0,1.0,BOB,221.0,BOB,198.0,23.0


In [96]:
#average rank
ordinal_avg_tourney = ordinal_tourney.groupby(["Season", "Team1", "Team2", "Result"]).mean()["RankDiff"].reset_index()
ordinal_avg_tourney.head(2)

,Season,Team1,Team2,Result,RankDiff
0,2010,1115.0,1181.0,0.0,225.333333
1,2010,1115.0,1457.0,1.0,37.541667


In [102]:
#"best" systems' + their average ranks - these systems were chosen by investigating the full logistic regression model,
#which included all systems as categoricals. Systems that had a p-value below .25 are included here.
ordinal_best_tourney = ordinal_tourney[ordinal_tourney["SystemName1"].isin(["ADE", "AP", "ATP", "BAR", "BNM", "CJB", "HAT", "HRN", "TBD", "USA"])]
ordinal_best_avg_tourney = ordinal_best_tourney.groupby(["Season", "Team1", "Team2", "Result"]).mean()["RankDiff"].reset_index()


In [103]:
#send to R for logistic regression
ordinal_tourney.to_csv("ordinal_tourney.csv")
ordinal_avg_tourney.to_csv("ordinal_avg_tourney.csv")
ordinal_best_tourney.to_csv("ordinal_best_tourney.csv")
ordinal_best_avg_tourney.to_csv("ordinal_best_avg_tourney.csv")

In investigating which ranking system is best, I found that taking the average of the ranking systems actually produces the best results.